<a href="https://www.kaggle.com/code/swabbie8/cabbage-type-vgg16?scriptVersionId=101472418" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install tf2onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.6/440.6 kB 1.3 MB/s eta 0:00:00


In [2]:
# importing the libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.model_selection import learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.metrics import confusion_matrix

In [3]:
# loading the directories 
training_dir = '../input/cabbagetypescnn/cabbage'

In [4]:
# getting the number of classes i.e. type of fruits
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 5


In [5]:
# importing the libraries
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications.vgg16 import VGG16
#from keras.preprocessing import image

IMAGE_SIZE = [150, 150]  # we will keep the image size as (64,64). You can increase the size for better results. 

# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG

# this will exclude the initial layers from training phase as there are already been trained.
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
#x = Dense(128, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = vgg.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



2022-07-22 08:44:20.204238: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 08:44:20.341798: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 08:44:20.342578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 08:44:20.343935: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

58900480/58889256 [==============================] - 0s 0us/step


In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [7]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=IMAGE_SIZE,
    batch_size=16,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    training_dir, # same directory as training data
    target_size=IMAGE_SIZE,
    batch_size=16,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 80 images belonging to 5 classes.
Found 20 images belonging to 5 classes.


In [8]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 16,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 16,
    epochs = 8)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-07-22 08:44:24.202946: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/8


2022-07-22 08:44:25.850991: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


5/5 [==============================] - 10s 593ms/step - loss: 1.7340 - accuracy: 0.2250 - val_loss: 1.3191 - val_accuracy: 0.5000
Epoch 2/8
5/5 [==============================] - 2s 343ms/step - loss: 1.0693 - accuracy: 0.6125 - val_loss: 1.1047 - val_accuracy: 0.4375
Epoch 3/8
5/5 [==============================] - 2s 337ms/step - loss: 0.6901 - accuracy: 0.8375 - val_loss: 0.8557 - val_accuracy: 0.7500
Epoch 4/8
5/5 [==============================] - 2s 340ms/step - loss: 0.4848 - accuracy: 0.9000 - val_loss: 0.6244 - val_accuracy: 0.9375
Epoch 5/8
5/5 [==============================] - 2s 342ms/step - loss: 0.3944 - accuracy: 0.8750 - val_loss: 0.5185 - val_accuracy: 0.8750
Epoch 6/8
5/5 [==============================] - 2s 336ms/step - loss: 0.2903 - accuracy: 0.9750 - val_loss: 0.5772 - val_accuracy: 0.8125
Epoch 7/8
5/5 [==============================] - 2s 307ms/step - loss: 0.2651 - accuracy: 0.9625 - val_loss: 0.5202 - val_accuracy: 0.8750
Epoch 8/8
5/5 [=====================

In [9]:
# The labels are stored in class_indices in dictionary form. 
# checking the labels
train_generator.class_indices

{'bokchoy': 0, 'green': 1, 'napa': 2, 'red': 3, 'savoy': 4}

In [10]:
print ('Training Accuracy = ' + str(history.history['accuracy']))
print ('Validation Accuracy = ' + str(history.history['val_accuracy']))

Training Accuracy = [0.22499999403953552, 0.612500011920929, 0.8374999761581421, 0.8999999761581421, 0.875, 0.9750000238418579, 0.9624999761581421, 1.0]
Validation Accuracy = [0.5, 0.4375, 0.75, 0.9375, 0.875, 0.8125, 0.875, 0.875]


In [11]:
model.save('model.h5')
#model.save('model.json')

In [12]:
import tensorflow as tf
import tf2onnx
import onnx

input_signature = [tf.TensorSpec([None ,150, 150, 3], tf.float32, name='x')]
# Use from_function for tf functions
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13)
onnx.save(onnx_model, "model.onnx")


2022-07-22 08:44:46.388636: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 08:44:46.389033: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-07-22 08:44:46.389172: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-07-22 08:44:46.389701: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 08:44:46.390083: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 08:44:46.390608: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

In [13]:
from tensorflow import keras
model = keras.models.load_model('model.h5')

In [14]:
from keras.preprocessing.image import load_img
image = load_img('../input/cabbagetypescnn/cabbage/red/13.jpg', target_size=(150, 150))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,150,150,3)
prediction = model.predict(img)

In [15]:
type(prediction)

numpy.ndarray

In [16]:
l={"bokchoy":prediction[0][0],"green":prediction[0][1],"napa":prediction[0][2],"red":prediction[0][3],"savoy":prediction[0][4]}
def get_key(val):
    for key, value in l.items():
         if val == value:
             return key
 
    return "key doesn't exist"

In [17]:
j=prediction.max()
get_key(j)

'red'